In [2]:
#| default_exp routes.role

In [3]:
#| export
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de


# Get and Search Routes

In [4]:
class Role_NotRetrieved(de.DomoError):
    def __init__(self,
                 domo_instance,
                 function_name,
                 status,
                 message,
                 role_id = None,
                 ):

        super().__init__(
            domo_instance=domo_instance,
            entity_id=role_id,
            function_name=function_name,
            status=status,
            message=message
        )


In [5]:
# | export
async def get_roles(auth: dmda.DomoAuth,
                    debug_api: bool = False,
                    session: httpx.AsyncClient = None,
                    ) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles'

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session=session
    )

    if len(res.response) == 0 or not res.is_success:
        raise Role_NotRetrieved(
            domo_instance=auth.domo_instance,
            function_name='get_roles',
            status=res.status,
            message = 'failed to retrieve roles')
    
    return res


#### sample implementation of get_roles

In [6]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_roles(auth=token_auth)

pd.DataFrame(res.response[0:5])


,id,name,description,authorityCount,userCount,created,modified
0,1,Admin,Full access to everything,77,15,1580425515000,1580425515000
1,2,Privileged,Full access except for editing users and setti...,34,313,1580425515000,1580425515000
2,3,Editor,"Can edit Cards, Pages, DataSets, and Dataflows",30,3,1580425515000,1580425515000
3,4,Participant,"Read only access on Cards, Pages DataSets and ...",6,3,1580425515000,1580425515000
4,5,Social,Access limited to social features,1,11,1580425515000,1580425515000


In [8]:
#| export


async def get_role_grants(auth: dmda.DomoAuth,
                          role_id: str,
                          debug_api: bool = False,
                          session : httpx.AsyncClient = None
                          ) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}/authorities'

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session = session
    )

    if len(res.response) == 0:
        raise Role_NotRetrieved(
            domo_instance= auth.domo_instance,
            function_name='get_role_grants',
            message=f"no grants returned for role {role_id}",
            status=res.status
        )

    return res

#### sample implementation of get_role_grants

In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_role_grants(auth=token_auth, role_id = 1)

print(res)
# pd.DataFrame(res.response[0:5])

ResponseGetData(status=200, response=['versions.deployment.manage', 'buzz.admin', 'dataflow.admin', 'goal.admin', 'dataset.admin', 'workflow.admin', 'profile.achieve.assign', 'datascience.notebooks.admin', 'content.dataapp.edit', 'dataset.export', 'user.invite', 'connector.admin', 'content.card.analyzerview', 'fileshare.create', 'customer.edit', 'content.variable.edit', 'pipeline.executor.job.admin', 'cloud.admin', 'versions.repository.admin', 'codeengine.package.manage', 'appstore.admin', 'datastore.admin', 'certifiedcontent.admin', 'group.admin', 'tasks.admin', 'group.edit', 'app.ddx', 'queue.create', 'domoapps.edit', 'content.export', 'codeengine.package.admin', 'domo.system.messages', 'publish.space.admin', 'authorization.roles', 'dataset.manage', 'goal.distribute', 'appstore.read', 'content.card.embed', 'content.admin', 'user.session.edit', 'cloud.override', 'datastore.create', 'user.edit', 'content.card.edit', 'saasaas.edit', 'developer.token.manage', 'publish.subscribers.manage'

In [ ]:
#| export
async def get_role_membership(auth: dmda.DomoAuth,
                              role_id: str,
                              debug_api: bool = False, 
                              session: httpx.AsyncClient = None,
                              return_raw : bool = False
                              ) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}/users'

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='GET',
        debug_api=debug_api,
        session=session
    )

    if len(res.response.get('users')) == 0:
        raise Role_NotRetrieved(
            domo_instance=auth.domo_instance,
            function_name='get_role_membership',
            message=f"no users returned for role {role_id}",
            status=res.status
        )

    if return_raw:
        return res
    
    res.response = res.response.get('users')
    
    return res

#### sample implementation of get_role_membership


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

res = await get_role_membership(auth=token_auth, role_id=1, return_raw = False)

pd.DataFrame(res.response[0:5])


,userId,email,displayName,roleId,department,jobTitle,hireDate,location,timeZone
0,1345408759,DataMaven42@gmail.com,Alexis Lorenz (DataMaven),1,Owner,Major Domo Support Specialist,1.590156e+12,NaN,NaN
1,1102391116,andrea.henderson@domo.com,Andrea Henderson,1,NaN,NaN,NaN,NaN,NaN
2,1095515127,andrew@crystalballers.ai,Andrew Chaffin,1,Crystal Ballers,Cofounder,NaN,,NaN
3,2074422502,bard.higgins@gmail.com,Bard Higgins,1,NaN,NaN,NaN,NaN,America/Denver
4,1865099622,Ben.Schein@domo.com,Ben Schein,1,NaN,NaN,NaN,NaN,NaN


# CRUD Routes

In [ ]:
#| export
async def create_role(auth: dmda.DomoAuth,
                      name: str,
                      description: str,
                      debug_api: bool = False) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles'

    body = {
        'name': name,
        'description': description
    }

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='POST',
        debug_api=debug_api,
        body=body
    )

    return res

In [ ]:
# | export
async def set_default_role(auth: dmda.DomoAuth,
                           role_id: str,
                           debug_api: bool = False,
                           session: httpx.AsyncClient = None) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/content/v1/customer-states/user.roleid.default'

    body = {
        'name': 'user.roleid.default',
        'value': role_id
    }

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        debug_api=debug_api,
        body=body,
        session=session
    )
    return res


In [ ]:
async def update_role_metadata(auth: dmda.DomoAuth,
                               role_id,
                               role_name,
                               role_description: str = None,
                               debug_api: bool = False
                               ):

    url = f"https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}"

    body = {"name": role_name,
            "description": role_description, "id": role_id}

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        debug_api=debug_api,
        body=body,
        session=session
    )

    
    return res


#### sample update_role_metadata

In [ ]:
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)



In [ ]:
#| export
async def update_role_grants(auth: dmda.DomoAuth,
                             role_id: str,
                             role_grant_list: list[str],
                             debug_api: bool = False) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}/authorities'

    if debug_api:
        print(url)

    res = await gd.get_data(auth=auth,
                         url=url,
                         method='PUT',
                         debug_api=debug_api, body=role_grant_list)
    return res

In [ ]:
#| export
async def role_membership_add_user(auth: dmda.DomoAuth,
                                   role_id: str,
                                   user_list: list[str],
                                   debug_api: bool = False) -> rgd.ResponseGetData:
    url = f'https://{auth.domo_instance}.domo.com/api/authorization/v1/roles/{role_id}/users'

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method='PUT',
        debug_api=debug_api,
        body=user_list
    )
    return res

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()
